**IMPORTS**

In [14]:
import json, re

**HELPER FUNCTIONS**

In [15]:
def getHTSRecord(path: str) -> list[dict[str, any]]:

    with open(path, 'r') as file:
        return json.loads(file.read())
    
def checkFinalQueryState(finalRecord: str, queryState: str) -> bool:

    if finalRecord == queryState:
        return True
    else:
        return False
    

**MAIN FUNCTIONS**

In [32]:
def queryRecord(query: list, htsRecord: list[dict[str, any]]) -> dict[str, any]:

    queryObj = {
        'query': query,
        'followup': [],
        'records': [],
        'finalRecord': query[len(query)-1]
    }
    indent = 0
    fIndex = -1
    currentIndex = 0
    limit = len(htsRecord) - 1
    match_followup = False
    run = True

    while run:

        for index, record in enumerate(htsRecord):
            
            if index == (limit):
                print('End of htsRecord')
                run = False
                break
            else:
                print(len(htsRecord), index)
                queryObj['followup'].append(htsRecord[index+1])
                fIndex += 1
            
            queryState = queryObj['query'][currentIndex]

            expression = re.compile(rf"{queryState}")

            #Case where the current record iterated matches the current queryState (even if it is not a full match)
            if 'htsno' in record and re.match(expression, record['htsno']):

                queryObj['records'].append(record)
                
                #Check if we are at the end of the query
                if checkFinalQueryState(queryObj['finalRecord'], queryState):

                    print(f"Found final record with description: {record['description']}")    
                    run = False
                    break
                
                ##Check for the queryObj.query and add necessary changes to the queryObj
                print(f'Found hts record with description: {record["description"]}')
                indent += 1
                currentIndex += 1

            #Case where the following record to an EH has an htsno key
            elif (record['indent'] == indent) and ('htsno' in queryObj['followup'][fIndex]):

                print(f'Found one EH with description: {record["description"]}')
                
                if re.match(expression, queryObj['followup'][fIndex]['htsno']):
                    queryObj['records'].append(record)
                    match_followup = True
                    print(f"Found following htsno after EH: {queryObj['followup'][fIndex-1]['description']}")

    return queryObj




In [30]:
query = ["0101", "0101.21", "0101.21.00", "0101.21.00.20"]
htsRecord = getHTSRecord('../../db_hts/temp/NEW_final_json_files/0101.json')

result = queryRecord(query, htsRecord)    

12 0
Found hts record with description: Live horses, asses, mules and hinnies:
12 1
Found one EH with description: Horses:
Found following htsno after EH: Horses:
12 2
Found HTS record matching after empty record Purebred breeding animals
Found hts record with description: Purebred breeding animals
12 3
12 4
Found final record with description: Females


In [31]:
result

{'query': ['0101', '0101.21', '0101.21.00', '0101.21.00.20'],
 'followup': [{'indent': 1, 'description': 'Horses:', 'superior': 'true'},
  {'htsno': '0101.21.00',
   'indent': 2,
   'description': 'Purebred breeding animals',
   'general': 'Free',
   'other': 'Free',
   'footnotes': [{'columns': ['general'],
     'marker': '1',
     'value': 'See 9903.88.15. ',
     'type': 'endnote'}]},
  {'htsno': '0101.21.00.10',
   'indent': 3,
   'description': 'Males',
   'units': ['No.']},
  {'htsno': '0101.21.00.20',
   'indent': 3,
   'description': 'Females',
   'units': ['No.']},
  {'htsno': '0101.29.00',
   'indent': 2,
   'description': 'Other',
   'general': 'Free',
   'other': '20%',
   'footnotes': [{'columns': ['general'],
     'marker': '1',
     'value': 'See 9903.88.15. ',
     'type': 'endnote'}]}],
 'records': [{'htsno': '0101',
   'indent': 0,
   'description': 'Live horses, asses, mules and hinnies:'},
  {'indent': 1, 'description': 'Horses:', 'superior': 'true'},
  {'htsno': '0